# Two Six Captial - Candidate Data Engineering Challenge

 Greg Netols

 Decemeber 14, 2017


In [0]:
import numpy as np
import pandas as pd

import locale
locale.setlocale( locale.LC_ALL, 'English_United States.1252')

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white')
sns.set(font_scale=1.25)


# 1 Parsing the files to dataframes

Even though all files were downloaded as CSV formatted files the 2013, 2014, and 2015 transaction files are in a tab delimited format. The 2016 transactions were in a true CSV format however there was a parse error on line 310 due to the line containing 8 columns instead of 5. After the row on line 310 was manually cleaned it was discovered that this was a problem through out the rest of the file. The error_bad_lines parameter was then set to False to get a print out of the remaining bad rows. Seeing that there were only 6 more lines that needed to be fixed it made sense to manually fix them rather than ignoring or creating a programatic way to handle them.


In [0]:
transactions2013 = pd.read_csv('C:\\FlatFiles\\TwoSix\\transactions_2013.csv', sep='\t')
transactions2014 = pd.read_csv('C:\\FlatFiles\\TwoSix\\transactions_2014.csv', sep='\t')
transactions2015 = pd.read_csv('C:\\FlatFiles\\TwoSix\\transactions_2015.csv', sep='\t')
transactions2016 = pd.read_csv('C:\\FlatFiles\\TwoSix\\transactions_2016.csv')


In [0]:
transactions2016.head()


# 2 Initial Data Cleaning

* Through visually looking at the text files it was obviouse that the column headers were not consistent across all files. The column names were adjusted for the 2013, 2014, 2015 files to bring them inline with the 2016 file. It was choosen to bring the column names in line with the 2016 file since column names with no spaces are preferred. This will now allow for the concatenatenation of all four files together making additional data cleaning steps easier.

* Striped unecessary whitespace from the three string columns (transaction_date, join_date, region).

* Date formating in the 2013, 2014, and 2015 file is not in a format that can be easily converted to a datetime datatype. A function was created that could be applied to the transaction_date and join_date column to change string months into numeric months. Completed the cleaning of the date columns by converting the datatype to datetime.

* The region column is defined as "The geographical region where the transaction was made" it is expected that this column is limited to a managable set of unique values. Which is true. There are 21 unique values and all are individual capital characters.

The goal of the inital data cleaning is to have numeric and date columns contrained to a data type. String values should be confirmed to be formatted in a predictable was or confirmed to occupy a set of expected values. In the end:

* user - constrained to an integer column and will not cause equivalency issues due to leading zeros or due to the integer being represented as a string.

* transaction_date - constrained to a datetime.

* sales_amount - constrained to a float64. I can be confident that there wil be no leading currency symbols, no string representations, and the values will be able to be aggregated.

* join_date - constrained to a datetime.

* region - limited to 21 unique values which is in line with the description of the column.



In [0]:
print( transactions2013.columns)
print( transactions2014.columns)
print( transactions2015.columns)
print( transactions2016.columns)


In [0]:
# Rename columns to be consistent accross all files
transactions2013.columns = ['user', 'transaction_date', 'sales_amount', 'join_date', 'region']
transactions2014.columns = ['user', 'transaction_date', 'sales_amount', 'join_date', 'region']
transactions2015.columns = ['user', 'transaction_date', 'sales_amount', 'join_date', 'region']


In [0]:
# Concatenate the 4 files to create a single transaction file
dataFrames = [transactions2013, transactions2014, transactions2015, transactions2016]
transactions = pd.concat(dataFrames)
transactions.head()


In [0]:
# remove any aditional whitespace in the three object(string) columns
transactions['transaction_date'] = transactions['transaction_date'].str.strip()
transactions['join_date'] = transactions['join_date'].str.strip()
transactions['region'] = transactions['region'].str.strip()


In [0]:
print(transactions.dtypes)


In [0]:
monthConversion = {'January':'01', 'February':'02', 'March':'03', 'April':'04', 'May':'05', 'June':'06',
                   'July':'07', 'August':'08', 'September':'09', 'October':'10', 'November':'11', 'December':'12'}

def convert_string_months(row):
    for stringMonth, numberMonth in monthConversion.items():
        row = row.replace(stringMonth, numberMonth)
    return row


In [0]:
# apply the convert_string_months to change written out month values
transactions['transaction_date'] = transactions['transaction_date'].astype(str).apply(convert_string_months)
transactions['join_date'] = transactions['join_date'].astype(str).apply(convert_string_months)


In [0]:
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'])
transactions['join_date'] = pd.to_datetime(transactions['join_date'])


In [0]:
transactions.head()


In [0]:
print(transactions.dtypes)


In [0]:
print( len(transactions['region'].unique()) )
print(transactions['region'].unique())


## 2.1 Additonal Cleaning - Null Clean up

Before any null values are handled there exists 3 sales_amounts that are null along with 11 join_dates.

* null sales amounts will be set to 0. It is possible that a user could make a tansaction without there being a sales amount. Perhaps the business released vouchers for free items which when used still created a transaction with a null sales amount.
* The join_date is defined as the 'date of the customer's first purchase' if it is null it will be populated with the earliest transaction_date that exists for that user.


In [0]:
transactions.isnull().sum(axis=0)


In [0]:
transactions[transactions['sales_amount'].isnull()]


In [0]:
transactions['sales_amount'] = transactions['sales_amount'].fillna(0)
transactions[transactions['sales_amount'].isnull()]


In [0]:
transactions[transactions['join_date'].isnull()]


In [0]:
transactions['min_transaction_date'] = transactions.groupby('user')['transaction_date'].transform('min')
transactions['join_date'] = transactions['join_date'].fillna(transactions['min_transaction_date'])
transactions = transactions.drop('min_transaction_date', axis=1)

transactions[transactions['join_date'].isnull()]


In [0]:
transactions.isnull().sum(axis=0)


## 2.2 Additonal Cleaning - Understanding the Data

At this point i have four questions that i want to answer before i begin extracting value from the data. These questions may inspire additional data cleaning steps that need to be performed.


### 2.2.1 - Are there any duplicate records?

I will need to decided if these should be considered duplicate events and removed, or if they are the user performing the same action twice in one day.


In [0]:
print(any(transactions.duplicated()))
print(sum(transactions.duplicated(keep=False)))


In [0]:
transactions[transactions.duplicated(['user', 'transaction_date', 'sales_amount'], keep=False)].sort_values(by=['user', 'transaction_date', 'sales_amount']).head(10)


There are definitly duplicates. However duplicates will not be removed. It is possible that two different items could be purchased by the same user, on the same date, and for the same amount.


### 2.2.2 - Are there any transaction dates that occur before the join date?
The join_date column is defined as "the date of a customer's first purchase" while the transaction_date is defined as "the date when the transaction was made". From the column definition there should be no join_dates that occur after the minimum transaction_date for a customer.


In [0]:
print( len(transactions[transactions['transaction_date'] < transactions['join_date']]) )


In [0]:
print( len(transactions[transactions['transaction_date'] >= transactions['join_date']]) )


There are 6990 records that contain a join date that occurs after the tansaction.



### 2.2.3 - Does each user have the same join_date across all of their transactions?    

Since join_date is defined as "The date of a customers first purchase" a customer should not have multiple join dates in the transaction files.


In [0]:
transactions[['user', 'join_date']].groupby(['user', 'join_date']).agg({'join_date': 'count'}).head(10)


From the above output it is visible that not all users have the same join_date accross all of their transactions. User 7 has their join dates divided amongst 2009-04-05 and 2009-05-04.


### 2.2.4 - Fixing the data issues found in 2.2.2 and 2.2.3

To bring the join_date column in line with its business definition join_date will be fixed in a two step process. If this is not resolved before the cohort analysis the cohort triangles will look more like cohort rectangles.

1. For each row set join_date equal to the minimum of join_date and transaction_date
2. For each user set their join_date for all transaction equal to their minimum join_date on the transaction file.

This will ensure that no rows will contain a join_date that comes after the transaction_date, and that each user has only one join_date.


In [0]:
transactions['join_date'] = transactions[['transaction_date', 'join_date']].min(axis=1)

transactions['join_date'] = transactions[['user', 'join_date']].groupby(['user']).transform('min')


In [0]:
print( len(transactions[transactions['transaction_date'] < transactions['join_date']]) )


In [0]:
transactions[['user', 'join_date']].groupby(['user', 'join_date']).agg({'join_date': 'count'}).head(10)


In [0]:
transactions[['user', 'join_date']].groupby(['user', 'join_date']).agg({'join_date': 'count'}).shape


Now their exists no transaction dates that come before the join_date. There are also no users that have more than one unique join_date accross their set of transactions.


### 2.2.5 - Are there any sales amounts that are negative?

These could indicate that the system is recording returns or adjustments. Should negative sales amounts contribute to revenue summations. This might depend on how strict of an accountant will be analyzing this report. For this challenge i will leave negative sales amounts unchanged.


In [0]:
len(transactions[transactions['sales_amount'] < 0])


In [0]:
transactions[transactions['sales_amount'] < 0].head()


There are negative sales amounts. I will be treating these as returns / adjustments for previous transactions and i will not be removing them.


# 3 Compute Yearly Time Series Metrics

a. Revenue

    Total sales during the year

b. Active User Count

    Total unique users that completed a transaction

c. Average Recenue Per Active User

    (Revenue) / (Active User Count)


In [0]:
timeSeries = transactions.copy()

groupedTimeSeries = timeSeries.groupby(timeSeries['transaction_date'].dt.year)

groupedTimeSeries = groupedTimeSeries.agg({'sales_amount': 'sum', 'user': 'nunique'})

groupedTimeSeries['Average Revenue Per Active User'] = groupedTimeSeries['sales_amount'] / groupedTimeSeries['user']


In [0]:
# rename columns and index for display purposes
groupedTimeSeries.index.name = 'Transaction Year'
groupedTimeSeries.columns = ['Revenue', 'Active User Count', 'Average Revenue Per Active User']


In [0]:
# Format dollar values
groupedTimeSeries['Revenue'] = groupedTimeSeries['Revenue'].map(locale.currency)
groupedTimeSeries['Average Revenue Per Active User'] = groupedTimeSeries['Average Revenue Per Active User'].map(locale.currency)


In [0]:
timeSeriesMetrics = groupedTimeSeries.rename_axis('Metrics', axis='columns').transpose()

timeSeriesMetrics


# 4 Compute Yearly New Users Joined


In [0]:
newUsers = transactions.copy()

newUsers = newUsers.groupby(newUsers['join_date'].dt.year)
newUsers = newUsers.agg({'user': 'nunique'})

newUsers.index.name = 'Year'
newUsers.columns = ['New Users Joined']
newUsers.index = newUsers.index.astype(int)


In [0]:
#Check that the sum of New Users Joined adds up to the number of unique users
print( newUsers['New Users Joined'].sum() )

print( len(transactions.user.unique()) )


In [0]:
newUsers


In [0]:
plt.figure(figsize=(14, 10))
ax = sns.barplot(newUsers.index, y=newUsers['New Users Joined'], data=newUsers, color = 'blue')
ax.set(xlabel='Year', ylabel='Count of New Users Joined', title='New Users Joined')
plt.show()


# 5 Cohort Analysis

Group users into cohorts based off of the year that they made their first purchase (join_date).


In [0]:
transactionCohort = transactions.copy()

transactionCohort['cohort_year'] = transactionCohort['join_date'].dt.year.astype(int)
transactionCohort['transaction_year'] = transactionCohort['transaction_date'].dt.year.astype(int)

transactionCohort.head()


Group transactions by the users cohort year and the transaction year. These groupings will be at the same granularity as the individual cells on the cohort triangle.


In [0]:
groupedCohort = transactionCohort.groupby(['cohort_year', 'transaction_year'])

cohorts = groupedCohort.agg({'sales_amount': 'sum',
                             'user': 'nunique'})

cohorts['average_revenue_per_active_user'] = cohorts['sales_amount'] / cohorts['user']

cohorts.columns = [ 'revenue', 'active_user_count', 'average_revenue_per_active_user']

cohorts.head(10)


To compute the cohort triangle for Average Revenue Per User the cummulative sum of users that have joined up until the transaction date will need to be known. This can easily be added to the data frame by joining the results from part 4 to the cohorts triangle data table.


In [0]:
cummulativeNewUsers = newUsers.copy()
cummulativeNewUsers['total_users'] = cummulativeNewUsers['New Users Joined'].cumsum()

cohortsJoined = pd.merge(cohorts.reset_index(), cummulativeNewUsers.reset_index(), how='left', left_on='cohort_year', right_on='Year').set_index(['cohort_year', 'transaction_year'])

cohortsJoined = cohortsJoined.drop('New Users Joined', axis=1)

cohortsJoined['average_revenue_per_user'] = cohortsJoined['revenue'] / cohortsJoined['total_users']

cohortsJoined['lifetime_value'] = cohortsJoined.reset_index().groupby(['cohort_year', 'transaction_year'])['average_revenue_per_user'].sum().groupby(level=[0]).cumsum()

cohortsJoined.head(10)


In [0]:
cohortsJoined['revenue'] 
cohortsJoined['average_revenue_per_active_user']
cohortsJoined['average_revenue_per_user'] 
cohortsJoined['lifetime_value'] 

cohortsJoined.head(10)


## 5.1 Revenue Cohort

Total revenue by year of first purchase c and by year of purchase t 


In [0]:
revenueCohort = cohortsJoined['revenue'].unstack(0).transpose()
revenueCohort


In [0]:
plt.figure(figsize=(16, 12))
plt.title('Revenue Cohort ($)')
htmp = sns.heatmap(revenueCohort, annot=True, fmt='g')
htmp.set(xlabel='Transaction Year', ylabel='Cohort Year')
plt.show()


## 5.2 Active User Count Cohort

Total revenue by year of first purchase c and by year of purchase t 


In [0]:
activeUserCountCohort = cohortsJoined['active_user_count'].unstack(0).transpose()
activeUserCountCohort


In [0]:
plt.figure(figsize=(16, 12))
plt.title('Active User Count Cohort')
htmp = sns.heatmap(activeUserCountCohort, annot=True, fmt='g')
htmp.set(xlabel='Transaction Year', ylabel='Cohort Year')
plt.show()


## 5.3 Average Revenue per Active User Cohort

Revenue (at year t, cohort year c) / Active User Count (at year t, cohort year c)


In [0]:
averageRevenueActiveUserCohort = cohortsJoined['average_revenue_per_active_user'].unstack(0).transpose()
averageRevenueActiveUserCohort


In [0]:
plt.figure(figsize=(16, 12))
plt.title('Average Revenue Per Active User Cohort ($)')
htmp = sns.heatmap(averageRevenueActiveUserCohort, annot=True, fmt='g')
htmp.set(xlabel='Transaction Year', ylabel='Cohort Year')
plt.show()


## 5.4 Average Revenue per User Cohort

Revenue (at year t, cohort year c) / New Users Joined (at cohort year c)


In [0]:
averageRevenueUserCohort = cohortsJoined['average_revenue_per_user'].unstack(0).transpose()
averageRevenueUserCohort


In [0]:
plt.figure(figsize=(16, 12))
plt.title('Average Revenue per User Cohort ($)')
htmp = sns.heatmap(averageRevenueUserCohort, annot=True, fmt='g')
htmp.set(xlabel='Transaction Year', ylabel='Cohort Year')
plt.show()


## 5.5 Lifetime Value Cohort

Cumulative Sum of Average Revenue Per User along cohort year c


In [0]:
lifeTimeValueCohort = cohortsJoined['lifetime_value'].unstack(0).transpose()
lifeTimeValueCohort


In [0]:
plt.figure(figsize=(16, 12))
plt.title('Life Time Value Cohort ($)')
htmp = sns.heatmap(lifeTimeValueCohort, annot=True, fmt='g')
htmp.set(xlabel='Transaction Year', ylabel='Cohort Year')
plt.show()


For the purpose of keeping the report concise i will not be doing the extra credit. However, building the reports for each region would be as simple as filtering the transactions using the following statement prior to building the tables and charts.

transactions[transactions['region'] == 'region value']

To create the reports for monthly groupings instead of yearly groupings would involve first representing month/year combinations as an integer formated as yyyymm. This integer representation of the year/month could then be used instead of the year when contructing the time series metrics, new users joined, and cohort triangles.


